In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [5]'.</span>

An Exception was encountered at 'In [5]'.

In [1]:
#This cell is tagged parameters for running this template using Papermill to inject values to overwrite these.
markdown = ""
section = 99
sub_pg = 99
end_nb = False
next_nb = ""


In [2]:
# Parameters
markdown = "\n# Part 4 of Helper Tools: Filtering Links\n\nThis page is primarily based on the following page at the Circos documentation site:\n\n\n\n- [4. Filtering Links](????????????)\n\n\n\nThat page is found as part number 4 of the ??? part ['Helper Tools' section](http://circos.ca/documentation/tutorials/quick_start/) of [the larger set of Circos tutorials](http://circos.ca/documentation/tutorials/).\n\n\n\nGo back to Part 3 by clicking [here &#8592;](HelperTools_Part3.ipynb).\n\n\n\n----\n\n\n\n9 --- Helper Tools\n==================\n\n4. Filtering Links\n------------------\n\nscript location\n---------------\n\n    tools/filterlinks\n\nscript usage\n------------\n\n    > cd tools/filterlinks\n    > ./run\n    Processing data/dog.vs.human.subset.txt\n    Now look in data/dog.vs.human.subset.filtered.txt\n\nThe script sends a new links file to STDOUT and the report shown above\nto STDERR.\n\nTo get the full manpage, use -man.\n\n    > cd tools/filterlinks\n    > bin/filterlinks -man\n\nAdjust the configuration file etc/filterlinks.conf to suit your needs.\n\ndetails\n-------\n\nThis script is used to generate a new link file, composed of links that\npass certain rules.\n\nAlthough Circos supports rules in its configuration file, which you can\nuse to alter data formatting on the fly, you may find that creating a\nnew link file is more convenient.\n\nFor example, if you are using Circos in an interactive web page, you\nwant to generate the image as quickly as possible. Since Circos\\'\nbuilt-in rule engine is slower than this script, filterlinks is handy\nway to eliminated unwanted data upstream of Circos.\n\nAnother situation in which filterlinks is useful is to generate an input\nfile to orderchr, which minimizes crossing of links by shuffling\nchromosomes around the image circle. Since the speed of orderchr\nproportional (in a nasty way) to the number of links, it\\'s important to\npass it as many links as you need to get the job done, but no more than\nthat.\n\nYou filter on any property of the link with filterlinks:\n\n-   link id\n-   chromosome\n-   start and end position\n-   size\n-   span (intersect with genomic region)\n-   format options (color, thickness, z, etc)\n\nand any property can be tested with one of these rules\n\n-   regular expression (e.g. chr1 matches \\\"x\\\")\n-   exact string match (e.g. chr1 is \\\"hs1\\\")\n-   smaller and greater than (numerical or string order) (e.g. start \\<\n    10Mb)\n-   integer span (e.g. link span intersects 10-20Mb)\n\nBriefly (read the manpage below for all the details), the results of\neach rule test are all OR\\'ed together. Thus, it is sufficient for one\nrule to pass to pass the link.\n\nThus, if you have three rules, the link will be passed if\n\n        RULE1 or RULE2 or RULE3\n\nBy assigning IDs to sets of rules, you can group rules together to build\nup a complex filter which includes AND. In the example below, rules 1-3\nhave been assigned a distinct ID (0) and rules 4-5 were given ID 1.\nThese IDs control how rule results are combined: intra-ID rules are\nAND\\'ed and inter-ID rules are OR\\'ed.\n\n      ( RULE1 and RULE2 and RULE3 ) OR ( RULE4 and RULE5 )\n        _________________________        _______________\n        ^ rule set 1 (e.g. ID=0)         ^ rule set 2 (e.g. ID=1)\n\nfilterlinks manpage\n-------------------\n\n-   [NAME](#NAME)\n-   [SYNOPSIS](#SYNOPSIS)\n-   [DESCRIPTION](#DESCRIPTION)\n-   [RULES](#RULES)\n    -   [Link Parameters](#Link_Parameters)\n    -   [Conditions](#Conditions)\n-   [EXAMPLES](#EXAMPLES)\n    -   [Filtering by Chromosomes](#Filtering_by_Chromosomes)\n    -   [Filtering by Position](#Filtering_by_Position)\n    -   [Filtering by Link Options](#Filtering_by_Link_Options)\n    -   [Mixing Conditions and IDs](#Mixing_Conditions_and_IDs)\n-   [HISTORY](#HISTORY)\n-   [BUGS](#BUGS)\n-   [AUTHOR](#AUTHOR)\n-   [CONTACT](#CONTACT)\n\n------------------------------------------------------------------------\n\n------------------------------------------------------------------------\n\n[NAME]{#NAME}\n=============\n\nfilterlinks - filter the link file based on link parameters\n\n------------------------------------------------------------------------\n\n[SYNOPSIS]{#SYNOPSIS}\n=====================\n\n      filterlinks -links linkfile.txt [-nointer] [-nointra] [-debug]\n\n------------------------------------------------------------------------\n\n[DESCRIPTION]{#DESCRIPTION}\n===========================\n\n------------------------------------------------------------------------\n\n[RULES]{#RULES}\n===============\n\nA filter rules contains two parts: the link parameter which is tested\nand a list of acceptable conditions.\n\nThe two exceptions are the -nointer and -nointra flags. These can be\nused to filter out inter-chromosomal links (ends of link are on\ndifferent chromosomes) and intra-chromosomal links (ends of link are on\nthe same chromosome). These two rules are strict, meaning that if a link\ndoes not pass them, no other rules are tested and the link is\nimmediately rejected.\n\n------------------------------------------------------------------------\n\n[Link Parameters]{#Link_Parameters}\n-----------------------------------\n\n      link_param = condition1,condition2,...\n\nBecause each link has two ends, each link parameter may give rise to\nthree distinct rules\n\n      link_param   = condition1,condition2,...\n      link_param_1 = condition1,condition2,...\n      link_param_2 = condition1,condition2,...\n\nwhich test, respectively, both ends with the condition (both ends must\npass), the first end, and the second end. The first end of the link\ncorresponds to the first line of the link line pair. For example, given\nthe link \\... link018136 cf12 9800000 9900000 link018136 hs6 37914056\n37916509 \\...\n\nthe first end is cf12:9800000-9900000 and the second end is\nhs6:37914056-37916509.\n\n-   **[chr]{#item_chr}**\n\n    Applies the condition to the chromosome of the link.\n\n          chr   = 1\n          chr_2 = x\n\n-   **[start, end, size]{#item_start}**\n\n    Applies the condition to the start, end or size of the link. The\n    link size is end-start+1.\n\n          start = [?<]10000000\n          end   = [?>]50000000\n\n-   **[span]{#item_span}**\n\n    Applies the condition to the span of the link and should be used\n    with the \\`\\`s\\'\\' condition TYPE.\n\n          span = [?i]1000-2000\n\n-   **[id]{#item_id}**\n    Applies the condition to the id of the link.\n-   **[color, thickness, z, etc.]{#item_color}**\n\n    Any condition that is not one of id, chr, start, end, size, span is\n    assumed to be a link option and is applied to the option of the\n    link. For example, options include color, thickness, and z.\n\n          color = [?e]chr12\n          z = [?>]10\n\n------------------------------------------------------------------------\n\n[Conditions]{#Conditions}\n-------------------------\n\nA condition has the following format\n\n      { [?TYPE {ID} {!} ] } CONDITION\n\nwhere elements in { } are optional. Briefly, TYPE is used to indicate\nhow the CONDITION text should be applied (e.g. regular expression,\ninteger range, exact match, etc). The ID is used to combine rules so\nthat their match status is AND\\'ed together to determine whether the\nlink passes. The trailing \\`\\`!\\'\\' is used to negate the rule (i.e. for\nthe link to pass, the rule must fail).\n\n-   **[Default Condition is a Regular Expression]{#item_Default}**\n\n    If no optional elements in the condition are specified, it is\n    treated as a regular expression. For example,\n\n          LINK_PARAM = 12\n\n    would apply the regular expression \\`\\`12\\'\\' to the link parameter.\n    You can provide a list of conditions with ;; as a delimiter (you can\n    adjust the delimiter in the configuration file).\n\n          LINK_PARAM = 12;;x;;y\n\n    which are interpreted as a series of regular expressions used to\n    test the link parameter. The link will be passed if ANY condition\n    matches (i.e. match results are OR\\'ed). If you want match results\n    to be AND\\'ed (i.e. multiple rules must match for the link to pass,\n    read below).\n\n    The regular expression is case-insensitive.\n\n-   **[Adjusting Condition Type]{#item_Adjusting}**\n\n    The following conditions types are possible\n\n    -   **[r - regular expression (default)]{#item_r}**\n\n              LINK_PARAM = 12\n              LINK_PARAM = 12;;x;;y\n\n        You can specify the type as a regular expression explicitly with\n        \\[?r\\] but this is not necessary because that is the default.\n\n              LINK_PARAM = [?r]12;;[?r]x;;[?r]y\n\n    -   **[s - integer span]{#item_s}**\n\n        The syntax of the integer range is any string supported by\n        Set::IntSpan.\n\n              LINK_PARAM = [?s]1000-2000\n              LINK_PARAM = [?s]1000-2000,3000-4000\n              LINK_PARAM = [?s]1000-2000,3000-)\n              LINK_PARAM = [?s](-1000,2000,3000-)\n              LINK_PARAM = [?s](-1000,2000,3000-4000,5000-)\n\n    -   **[e - exact match]{#item_e}**\n\n        The exact match is useful for matching chromosome names in cases\n        where regular expressions might match other chromosomes (and you\n        don\\'t want to include anchors in your regular expression).\n\n              LINK_PARAM = [?e]chr1\n              LINK_PARAM = [?e]chr1;;[?e]chr2\n\n        Note that the condition type must be prefixed to each individual\n        condition, if a list of conditions is supplied.\n\n        The exact match is not case-sensitive.\n\n    -   **[\\< - less than]{#item__less_than}**\n\n        If the value is a number, numerical \\< is used, otherwise the\n        values are compared based on asciibetic order (e.g. le).\n\n              # LINK_PARAM must be less than 100\n              LINK_PARAM = [?<]100\n\n              # LINK_PARAM must be less (in the asciibetic sense) than chr20 (e.g. chr1, chr11, chr111, chr19, etc)\n              LINK_PARAM = [?<]chr20\n\n    -   **[\\> - greater than]{#item__greater_than}**\n        Works just like the less than condition \\[?\\<\\].\n    -   **[mixing condition types]{#item_mixing}**\n\n        You can have multiple condition types for a parameter. Remember\n        that results of each condition will be OR\\'ed together.\n\n              LINK_PARAM = 1,[?e]chr5,[?e]chr22\n\n        The first condition is a regular expression (by default). The\n        second and third conditions are exact text matches for chr5 and\n        chr22. Thus, the LINK\\_PARAM will pass if (a) it contains a\n        \\`\\`1\\'\\', or (b) it is \\`\\`chr5\\'\\' or (c) it is \\`\\`chr22\\'\\'.\n\n-   **[Negating a Condition]{#item_Negating}**\n\n    In order to negate a condition, use \\`\\`!\\'\\'. When \\`\\`!\\'\\' is\n    used, the condition must fail for the result to be acceptable.\n\n          # must not match regular expression \"1\"\n          LINK_PARAM = [?r!]1\n\n          # must not be \"chr12\"\n          LINK_PARAM = [?e!]chr12\n\n          # must not be within the range 1000-2000\n          LINK_PARAM = [?i!]1000-2000\n\n    In order to combine negated conditions with positive ones, you will\n    need to group conditions so that their results are AND\\'ed.\n\n-   **[Grouping Conditions]{#item_Grouping}**\n\n    So far, all condition results were evaluated with OR. In other\n    words, if you had a list of conditions, the successful pass of any\n    of the conditions resulted in the link being passed. This is useful\n    if you want to accept multiple values\n\n          # chr12 or chr14 \n          LINK_PARAM = [?e]chr12;;[?e]chr14\n\n    However, what if you wanted to match regular expression \\`\\`1\\'\\'\n    but not chr14. Here\\'s where the ID field comes in. By tagging\n    multiple conditions with the same ID field the results of each of\n    these conditions is AND\\'ed together to determine whether the link\n    passes.\n\n          # ID=0 \n          # match regular expression \"1\" AND not be \"chr14\"\n          LINK_PARAM = [?r0]1;;[?e0!]chr14\n\n------------------------------------------------------------------------\n\n[EXAMPLES]{#EXAMPLES}\n=====================\n\nBelow are some examples to get you started. Note the interplay between\nconditions with IDs and condition without IDs. The former collate\nconditions into AND\\'ed sets, which are then in turn OR\\'ed with other\nsets and with conditions without IDs.\n\n------------------------------------------------------------------------\n\n[Filtering by Chromosomes]{#Filtering_by_Chromosomes}\n-----------------------------------------------------\n\nTo select links in which both ends match regular expression \\`\\`1\\'\\'\n\n      chr = 1\n\nSo simple. Now, to select links in with either ends matches regular\nexpression \\`\\`1\\'\\',\n\n      chr_1 = 1\n      chr_2 = 1\n\nThe difference between these two cases is that in the first instance,\nsince the link\\_parameter does not include a \\_1 or \\_2 suffix, the\ncondition is applied to both ends of the link and both ends must pass.\nIn the second case, each end is tested independently and the results are\nOR\\'ed together.\n\nIf you want links where the first chromosome matches x or the second\nmatches y,\n\n      chr_1 = x\n      chr_2 = y\n\nThe test is (chr\\_1 match \\`\\`x\\'\\') OR (chr\\_2 match \\`\\`y\\'\\'). Note,\nhowever, that this set of rules requires that the first chromosome match\n\\`\\`x\\'\\' OR the second chromosome match \\`\\`y\\'\\'. It will fail if the\nfirst chromosome matches \\`\\`y\\'\\' and the second matches \\`\\`x\\'\\'. To\nmatch both possibilities, you might try\n\n      chr_1 = x;;y\n      chr_2 = y;;x\n\nIn this case the test is (chr\\_1 match \\`\\`x\\'\\') OR (chr\\_1 match\n\\`\\`y\\'\\') OR (chr\\_2 match \\`\\`x\\'\\') OR (chr\\_2 match \\`\\`y\\'\\').\n\nIf you are looking for links between x and y chromosomes, then you\nrequire the results of each condition to be AND\\'ed. For this, use IDs\n\n      chr_1 = [?r1]x\n      chr_2 = [?r1]y\n\nBoth of these rules have ID=1 and are therefore grouped into a set.\nMatch results within a set are AND\\'ed. Thus, the test is (chr\\_1 match\n\\`\\`x\\'\\') AND (chr\\_2 match \\`\\`y\\'\\'). If you want to match the other\norder too,\n\n      chr_1 = [?r1]x;;[?r2]y\n      chr_2 = [?r1]y;;[?r2]x\n\nIn this example, there are two IDs. The rules with ID=0 match chr1 to\n\\`\\`x\\'\\' and chr2 to \\`\\`y\\'\\' and the rules with ID=1 match the\nconverse (chr1 to \\`\\`y\\'\\' and chr2 to \\`\\`x\\'\\').\n\nNow let\\'s suppose we want links that are either cf1-hs6, cf14-hs7 or\ncfx-hsx. Here cf is a dog chromosome and hs is a human chromosome. The\nrule for this is\n\n      chr_1 = [?e1]cf1;;[?e2]cf14;;[?e3]cfx\n      chr_2 = [?e1]hs6;;[?e2]hs7;;[?e3]hsx\n\nYou can add additional conditions without IDs to accept more links. For\nexample, if you also wanted to add any links for which chr\\_1 was cf9 or\nfor which chr\\_2 matched \\`\\`3\\'\\'\n\n      chr_1 = [?e1]cf1;;[?e2]cf14;;[?e3]cfx;;[?e]cf9\n      chr_2 = [?e1]hs6;;[?e2]hs7;;[?e3]hsx;;3\n\nRemember that \\[?r\\]3 is the same as 3, since the default condition type\nis a regular expression.\n\nYou can take advantage of the \\`\\`!\\'\\' flag to negate rules to avoid\nchromosomes. For example, if you want links between cfx and any\nchromosome other than hsx\n\n      chr_1 = [?e1]cfx\n      chr_2 = [?e1!]hsx\n\nand here the test is (chr\\_1 is cfx) AND (chr\\_2 is not hsx).\n\nYou can combine chr with chr\\_1/chr\\_2 rules\n\n      chr   = 2\n      chr_1 = [?e1]cfx\n      chr_2 = [?e1!]hsx\n\nto produce the test ( (chr\\_1 is cfx) AND (chr\\_2 is not hsx) ) OR (\nchr\\_1 matches \\`\\`2\\'\\' AND chr\\_2 matches \\`\\`2\\'\\' ). Use \\`\\`chr\\'\\'\nas the parameter if you want to apply the same condition to both ends of\nth elink and chr\\_1 and chr\\_2 to apply different conditions.\n\n------------------------------------------------------------------------\n\n[Filtering by Position]{#Filtering_by_Position}\n-----------------------------------------------\n\nTo test link position, use the parameters \\`\\`start\\'\\', \\`\\`end\\'\\' and\n\\`\\`span\\'\\'. Both \\`\\`start\\'\\' and \\`\\`end\\'\\' are ideal for testing\nwith condition type \\< and \\>. To select links for which both ends start\nbefore 10,000,000\n\n      start = [?<]1e7\n      # or\n      start = [?<]10000000\n\nto add another OR\\'ed condition to pass links with start values beyond\n100,000,000\n\n      start = [?<]1e7;;[?>]1e8\n\nA more complex test for the \\`\\`start\\'\\' and \\`\\`end\\'\\' values can be\nleveled by using the \\`\\`s\\'\\' condition type, which tests for\nmembership within a span. This rule\n\n      start = [?i]1e6-2e6,3e6-4e6\n\nwill pass links for which both ends are within 1-2Mb or 3-4Mb. Note that\nthe \\`\\`,\\'\\' in this condition is part of the span and does not create\na new condition. To have two conditions, use the ;; delimiter.\n\n      start = [?i]1e6-2e6,3e6-4e6;;[?s]1e7-1.1e7,3e6-4e6\n\nWhen using the \\`\\`span\\'\\' parameter, you should always use the\n\\`\\`s\\'\\' condition type. This will check whether the link span\nintersects the provided span.\n\n      span = [?s]2e7-5e7\n\nThis will select all links whose spans (at both ends) intersect the\ncoordinates 20-50Mb. To be more selective, use the \\_1 and \\_2 suffixes.\n\n      span_1 = [?s1]2e7-5e7\n      span_2 = [?s1]2e7-2.5e7\n\nwill select links joining 20-50Mb regions to 20-25Mb regions. An ID was\nrequired here to make the results AND\\'ed. To avoid certain regions, use\nthe \\`\\`!\\'\\' flag\n\n      span = [?s!](-1e7\n\nwill avoid all links within the first 10Mb.\n\n------------------------------------------------------------------------\n\n[Filtering by Link Options]{#Filtering_by_Link_Options}\n-------------------------------------------------------\n\nAny link option such as \\`\\`color\\'\\', \\`\\`thickness\\'\\', or \\`\\`z\\'\\'\ncan be tested in similar rules.\n\n      # links with z value greater than 10\n      z = [?>]10 \n\n      # links with z value between 5 and 15\n      z = [?s]5-15\n\n------------------------------------------------------------------------\n\n[Mixing Conditions and IDs]{#Mixing_Conditions_and_IDs}\n-------------------------------------------------------\n\nYou can write fairly complex rules by combining different link\nparameter, rule types and IDs.\n\nFor example to apply the following filter\n\n      (\n      between (hs1 and cf6) \n      AND\n      within 75-80 Mb on hs1\n      AND \n      larger than 5kb on hs1\n      )\n\n      OR\n\n      (\n      larger than 500kb on hs1\n      )\n\nuse the following rules\n\n      chr_1   = [?e1]cf6\n      chr_2   = [?e1]hs1\n      span_2  = [?s1]75e6-80e6\n      size_2  = [?>1]5e3;;[?>]500e3\n\n------------------------------------------------------------------------\n\n[HISTORY]{#HISTORY}\n===================\n\n-   **[23 July 2008]{#item_23}**\n    Reworked rules and conditions to include TYPE and ID.\n-   **[9 July 2008]{#item_9}**\n\n    Started and versioned.\n\n------------------------------------------------------------------------\n\n[BUGS]{#BUGS}\n=============\n\n------------------------------------------------------------------------\n\n[AUTHOR]{#AUTHOR}\n=================\n\nMartin Krzywinski\n\n------------------------------------------------------------------------\n\n[CONTACT]{#CONTACT}\n===================\n\n      Martin Krzywinski\n      Genome Sciences Centre\n      Vancouver BC Canada\n      www.bcgsc.ca\n      martink@bcgsc.ca\n\n"
section = 9
sub_pg = 4
end_nb = False
next_nb = "HelperTools_Part5.ipynb"


In [3]:
'''from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
printmd(markdown)'''
print(markdown) #use this because want to be able to edit the markdown and rendering it with `printmd` diables that


# Part 4 of Helper Tools: Filtering Links

This page is primarily based on the following page at the Circos documentation site:



- [4. Filtering Links](????????????)



That page is found as part number 4 of the ??? part ['Helper Tools' section](http://circos.ca/documentation/tutorials/quick_start/) of [the larger set of Circos tutorials](http://circos.ca/documentation/tutorials/).



Go back to Part 3 by clicking [here &#8592;](HelperTools_Part3.ipynb).



----



9 --- Helper Tools

4. Filtering Links
------------------

script location
---------------

    tools/filterlinks

script usage
------------

    > cd tools/filterlinks
    > ./run
    Processing data/dog.vs.human.subset.txt
    Now look in data/dog.vs.human.subset.filtered.txt

The script sends a new links file to STDOUT and the report shown above
to STDERR.

To get the full manpage, use -man.

    > cd tools/filterlinks
    > bin/filterlinks -man

Adjust the configuration file etc/filterlinks.conf to suit your needs.

d



----

### Generating the plot produced by this example code

The following two cells will generate the plot. The first cell adjusts the current working directory.

In [4]:
%cd ../circos-tutorials-0.67/tutorials/{section}/{sub_pg}/

/home/jovyan/circos-tutorials-0.67/tutorials/9/4


In [5]:
%%bash
../../../../circos-0.69-6/bin/circos -conf circos.conf

debuggroup summary 0.21s welcome to circos v0.69-6 31 July 2017 on Perl 5.022000
debuggroup summary 0.21s current working directory /home/jovyan/circos-tutorials-0.67/tutorials/9/4
debuggroup summary 0.21s command ../../../../circos-0.69-6/bin/circos -conf circos.conf
debuggroup summary 0.21s loading configuration from file circos.conf

  *** CIRCOS ERROR ***

      cwd: /home/jovyan/circos-tutorials-0.67/tutorials/9/4

      command: ../../../../circos-0.69-6/bin/circos -conf circos.conf

  CONFIGURATION FILE ERROR

  Circos could not find the configuration file [circos.conf]. To run Circos, you
  need to specify this file using the -conf flag. The configuration file
  contains all the parameters that define the image, including input files,
  image size, formatting, etc.

  If you do not use the -conf flag, Circos will attempt to look for a file
  circos.conf in several reasonable places such as . etc/ ../etc

  To see where Circos looks for the file, use

      circos -debug_flag io

Use of uninitialized value in subroutine entry at /home/jovyan/circos-0.69-6/bin/../lib/Circos/Configuration.pm line 781.
 at /home/jovyan/circos-0.69-6/bin/../lib/Circos/Error.pm line 425.
	Circos::Error::fatal_error("configuration", "missing", "circos.conf") called at /home/jovyan/circos-0.69-6/bin/../lib/Circos/Configuration.pm line 796
	Circos::Configuration::loadconfiguration("circos.conf") called at /home/jovyan/circos-0.69-6/bin/../lib/Circos.pm line 148
	Circos::run("Circos", "_cwd", "/home/jovyan/circos-tutorials-0.67/tutorials/9/4", "configfile", "circos.conf", "_argv", "-conf circos.conf") called at ../../../../circos-0.69-6/bin/circos line 536


CalledProcessError: Command 'b'../../../../circos-0.69-6/bin/circos -conf circos.conf\n'' returned non-zero exit status 255.

View the plot in this page using the following cell.

In [ ]:
from IPython.display import Image
Image("circos.png")

----

In [ ]:
next_md = ("Continue on to the next part of the Quick Start series by clicking [here &#10145;]({})</font>.".format(next_nb))
if end_nb:
    next_md = ("End of tutorials.")
#printmd(next_md)
print(next_md) #use this because want to be able to edit the markdown and rendering it with `printmd` diables that

----